In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely import wkt
from shapely.ops import cascaded_union


import matplotlib.pyplot as plt
import geopandas as gpd
import pysal as ps

#plot map of grunnkrets
def plot_gk():
    df = pd.read_csv('../data//grunnkrets_norway_stripped.csv')
    df = gpd.GeoDataFrame(df)
    df['geometry'] = gpd.GeoSeries.from_wkt(df['geometry'])
    df = gpd.GeoDataFrame(df, geometry='geometry')
    df.plot(figsize=(20, 20))
    plt.show()

    #plot map of busstops
    df = pd.read_csv('../data//busstops_norway.csv')
    df = gpd.GeoDataFrame(df)
    df['geometry'] = gpd.GeoSeries.from_wkt(df['geometry'])
    df = gpd.GeoDataFrame(df, geometry='geometry')
    df.plot(figsize=(20, 20))
    plt.show()



train_data = pd.read_csv('../data//stores_train.csv', nrows= 1000)
place = pd.read_csv('../data//plaace_hierarchy.csv', nrows= 1000)
train_data = pd.merge(train_data, place[['plaace_hierarchy_id', 'lv1', 'lv2', 'lv3']], on='plaace_hierarchy_id', how='outer')
train_data.head()


In [ ]:
#number of stores within x radius
import math
import geopy.distance


df = train_data
df=df.loc[:,['store_id','lat','lon', 'lv2']]

df['num_stores_within_distance'] = 0
def stores_within_distance(distance):
    for index in range(len(df)):
        lvl1 = df._get_value(index,'lv2')
        lat1 = df._get_value(index,'lat')
        lon1 = df._get_value(index,'lon')
        count = 0
        for inx in range(len(df)):
            lvl2 = df._get_value(inx,'lv2')
            lat2 = df._get_value(inx,'lat')
            lon2 = df._get_value(inx,'lon')
            try:
                dist = geopy.distance.geodesic((lat1, lon1), (lat2, lon2)).km #some values have nan in them. must be fixed. 
                print(lat1, lat2, lon1, lon2)
            except:
                print(lat1, lat2, lon1, lon2)

            if dist < distance and lvl1 == lvl2:
                count += 1
        df._set_value(index, 'num_stores_within_distance', count- 1)


#stores_within_distance(0.1)



#df.head(20)


In [ ]:
#distance from shop to closest buss station
from dis import dis
import math
import geopandas
import numpy
from shapely import wkt
from shapely import wkb

#busstops
gdf = gpd.GeoDataFrame(pd.read_csv('../data//busstops_norway.csv', nrows=100))
gdf=gdf.loc[:,['importance_level', 'geometry']]
gdf['geometry'] = gpd.GeoSeries.from_wkt(gdf['geometry'])

# stores train
df['distance_to_closest_buss_station'] = 0
df['importance_level'] = 0
df['closest_local_knutepunkt'] = 0
df = train_data

#add to dataframe
for index in range(len(gdf)):
    closest = 100000 #large number
    importance = ""
    closest_high_importance = 0
    lat = df._get_value(index,'lat')
    lon = df._get_value(index,'lon')
    for numbuss in range(len(gdf)):
        val = gdf._get_value(index,'geometry')
        dist = geopy.distance.geodesic((val.x, val.y), (lat, lon)).km
        if dist < closest:
            closest = dist
            importance = gdf._get_value(index,'importance_level')
        if gdf._get_value(index,'importance_level') == 'Lokalt knutepunkt':
            closest_high_importance = dist
    df._set_value(index, 'distance_to_closest_buss_station', closest)
    df._set_value(index, 'importance_level', importance)
    df._set_value(index, 'closest_local_knutepunkt', closest_high_importance)

df.head()






In [ ]:
# make postal code per store 

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")


# Display
train_data = pd.read_csv('../data//stores_train.csv', nrows= 10)
df = train_data
df['postcode'] = 0
for index in range(len(df)):
    lat = df._get_value(index,'lat')
    lon = df._get_value(index,'lon')
    location = geolocator.reverse(str(lat)+","+str(lon))
    address = location.raw['address']
    zipcode = address.get('postcode')
    df._set_value(index, 'postcode', str(zipcode))
df['postcode']= df.postcode.astype('category')
df.head(10)